In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2

In [ ]:
device_numbers = [
    '21187339', 
    # '21187335', 
    '21174907'
    ]  # 21187339 is master

pre = 'headtracking_2023-05-03_09-26-29'
pre = '../headtracking_2023-05-04_09-57-20'
pre = '../headtracking_2023-05-04_11-45-20'
# pre = '../out/headtracking_2023-05-08_10-32-56'
get_device_text_path = lambda device_number: f"{pre}_{device_number}.txt"
get_device_video_path = lambda device: f"{pre}_{device}-0000.avi"
get_device_video_path = lambda device, count: f"{pre}_{device}/f{count:09d}.jpg"

def get_data(device, t_sys=False):
    with open(get_device_text_path(device)) as fh:
        def _process_line(line):
            try:
                a, b, c = line.strip().split(',')
                return int(a), float(c) if t_sys else float(b)
            except:
                a, b = line.strip().split(',')
                return int(a), float(b)
        return dict(_process_line(line) for line in fh)

In [ ]:
data = [get_data(device) for device in device_numbers]
df = pd.DataFrame(data).T
df


In [ ]:
data = [get_data(device, True) for device in device_numbers]
df = pd.DataFrame(data).T
df

In [ ]:
plt.plot(df.loc[130:280], label=[device_numbers[i] for i in df.columns])
plt.ylabel('time [ms]')
plt.xlabel('frame')
plt.title('frame drops')
plt.legend()

In [ ]:
def all_close(times):
    a, b, c = times
    return abs(a-b) < 8 and abs(a-c) < 8

def get_nearby(df, i, ref_cam=0):
    minima = ((df - df.loc[i, ref_cam])**2).idxmin()
    times = [df.loc[minima[j], j] for j in range(len(device_numbers))]
    return minima, times

minima, times = get_nearby(df, 1000)

# samples = np.random.choice(range(len(df)), 3)
# sum(all_close(get_nearby(df, i)[1]) for i in samples)

In [ ]:
# minima, times = get_nearby(df, 51693)
minima, times = get_nearby(df, 373)
minima, times = get_nearby(df, 1)

print(minima)
print(times)

fig, ax = plt.subplots(len(device_numbers))
fig.set_size_inches(30,20)
for i in range(len(device_numbers)):
    frame_number = minima[i]
    # cap = cv2.VideoCapture(get_device_video_path(device_numbers[i]))
    # cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
    # res, frame = cap.read()
    frame = cv2.imread(get_device_video_path(device_numbers[i], frame_number))
    ax[i].set_title(f'frame: {frame_number} time: {times[i]}')
    ax[i].imshow(frame)
    ax[i].axis('off')

In [ ]:
video_paths = [
    'primary-05082023113730-0000.avi',
    'secondary-05082023113723-0000.avi',
    'third-05082023113738-0000.avi'
]
fig, ax = plt.subplots(len(video_paths))
fig.set_size_inches(30,20)
for i, fn in enumerate(video_paths):
    frame_number = 300
    cap = cv2.VideoCapture(fn)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
    res, frame = cap.read()
    ax[i].imshow(frame)
    ax[i].axis('off')